In [ ]:
# Josephson Junction coupling
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
from itertools import repeat
import seaborn as sn

alpha = 0.02
alpha_array_inner_layer = []
alpha_array_outer_layer = []
beta_array = []
avg_error_func_array = []
beta = 0
alpha_max = 0.03
beta_max = 0.01
beta_repeat_times = int((beta_max-beta)/0.0005)+1

while beta <= beta_max:
    while alpha <= alpha_max:
        alpha_array_inner_layer.append(alpha)
        alpha += 0.0005
    beta_array.extend(repeat(beta,beta_repeat_times))
    alpha_array_outer_layer.extend(alpha_array_inner_layer)
    beta += 0.0005

xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

tStart = 0
tEnd = 2000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]

k = 0
while k < len(beta_array) and k < len(alpha_array_outer_layer):
    alpha = alpha_array_outer_layer[k]
    beta = beta_array[k]
    
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))
    
    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]

    # average of error function
    sum_error_func = 0
    i = 0
    while i<len(t_sol):
        sum_error_func += math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2)
        i += 1
    avg_error_func = sum_error_func/len(t_sol)
    avg_error_func_array.append(avg_error_func)
    k += 1

# print(len(avg_error_func_array))
avg_error_func_array_for_matrix = np.array_split(avg_error_func_array, 20)
print(avg_error_func_array_for_matrix)
con = np.vstack((avg_error_func_array_for_matrix))
sn.set(rc = {'figure.figsize':(15, 15)})
heatmap = sn.heatmap(data=con)
# tick labels
n_xticklabels = 0.02
list_xticklabels = []
while n_xticklabels <= 0.03:
    list_xticklabels.append('{:,}'.format(round(n_xticklabels, 5)))
    n_xticklabels += 0.0005
n_yticklabels = 0
list_yticklabels = []
while n_yticklabels <= 0.01:
    list_yticklabels.append('{:,}'.format(round(n_yticklabels, 5)))
    n_yticklabels += 0.0005
heatmap.set_xticklabels(list_xticklabels)
heatmap.set_yticklabels(list_yticklabels)
heatmap.invert_yaxis()
print(heatmap)